In [18]:
import time
import multiprocessing

def foobar(idx, lock):
    for i in range(0, 4):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx

def wrapper_factory(func, args, kwargs, internal_params):

    internal_args, internal_kwargs = internal_params
    
    lock, shared_memory_name = internal_args
    sm = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=False, size=1)
    smb = sm.buf
    
    #print("------")
    #print(func)
    #print(args)
    #print(kwargs)
    #print(internal_args)
    #print(internal_kwargs)

    try:
        return func(*args, **kwargs)
    finally:
        lock.acquire()
        smb[0] = int(smb[0]) + 1
        lock.release()

In [ ]:
import io
import sys

def redirect_stdout():
    sys.stdout = io.StringIO()
    

In [36]:
import multiprocessing
from tqdm.notebook import tqdm
import time

class MultiProcessor():

    def __init__(self, num_workers = 4):
        self.manager = multiprocessing.Manager()
        self.lock = self.manager.Lock()
        self.pool = multiprocessing.Pool(processes=4)
        self.shared_memory_name = "multiprocessor"
        try:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=True, size=1)
        except FileExistsError as fee:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=False, size=1)

        self.shared_memory_buffer = self.shared_memory.buf
        self.shared_memory_buffer[0] = 0

        self.progress_bar = None
        

    def _create_progress_bar(self, num_ops):

        return tqdm(range(num_ops))
        
    def _update_progress_bar(self, i):
        self.progress_bar.last_print_n = i
        self.progress_bar.n = i
        self.progress_bar.set_description(f'Completed Ops')
        self.progress_bar.refresh()

        
        
        
    def parralelize(self, func, arg_set, kwarg_set):    
        num_ops = len(arg_set)
        self.progress_bar = self._create_progress_bar(num_ops)
        self.shared_memory_buffer[0] = 0
        
        func_set = [func for i in range(0, num_ops)]
        internal_param_set = [([self.lock, self.shared_memory_name], {}) for i in range(0, num_ops)]
        
        param_set = zip(func_set, arg_set, kwarg_set, internal_param_set)
        
        apply_result = self.pool.starmap_async(wrapper_factory, param_set)
        
        while not apply_result.ready():
            self._update_progress_bar(self.shared_memory_buffer[0])
            time.sleep(0.5)
        self._update_progress_bar(self.shared_memory_buffer[0])

        return apply_result
        
        


In [37]:
from itertools import repeat

num_ops = 10
m = multiprocessing.Manager()
l = m.Lock()
arg_set = list(zip(list(range(0, num_ops)), repeat(l)))
kwarg_set = [{} for i in range(0, num_ops)]

mp = MultiProcessor(num_workers=1)
apply_result = mp.parralelize(foobar, arg_set, kwarg_set)

  0%|          | 0/10 [00:00<?, ?it/s]

1 -> 0
2 -> 0
3 -> 0
0 -> 0
1 -> 1
2 -> 1
3 -> 1
0 -> 1
1 -> 2
2 -> 2
3 -> 2
0 -> 2
1 -> 3
2 -> 3
3 -> 3
0 -> 3
4 -> 0
5 -> 0
6 -> 0
7 -> 0
4 -> 1
5 -> 1
6 -> 1
7 -> 1
4 -> 2
6 -> 2
5 -> 2
7 -> 2
4 -> 3
6 -> 3
5 -> 3
7 -> 3
8 -> 0
9 -> 0
8 -> 1
9 -> 1
8 -> 2
9 -> 2
8 -> 3
9 -> 3


In [ ]:
# dir(apply_result)

In [22]:
apply_result.get()

RuntimeError: Lock objects should only be shared between processes through inheritance